<a href="https://colab.research.google.com/github/Aman-Tamgadge/It-Analysis/blob/main/IT_Expenditure_Variance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
import io

# --------------------------------------------------------------------------------
# Step 1: Data Loading in Colab
# --------------------------------------------------------------------------------

# NOTE: The provided 'edit' link is for viewing the Google Sheet.
# To load it directly as raw data (CSV) in Colab, you must:
# 1. Ensure the Google Sheet is shared publicly (File > Share > Publish to web).
# 2. Get the 'CSV export link'.
#
# For the link you provided:
# https://docs.google.com/spreadsheets/d/1dcNLCA1otBvt08nB-DnMwggf6RJ-qsVvdz_wOkMF-9w/edit?usp=sharing
#
# Replace the '/edit?usp=sharing' part with '/export?format=csv&gid=0'
# (Assuming your data is on the first sheet, which has gid=0)
#
# Using the derived CSV URL:
csv_url = "https://docs.google.com/spreadsheets/d/1dcNLCA1otBvt08nB-DnMwggf6RJ-qsVvdz_wOkMF-9w/export?format=csv&gid=0"

try:
    df = pd.read_csv(csv_url)
    print("Data loaded successfully.")
except Exception as e:
    print(f"Error loading data. Please check the CSV URL and permissions: {e}")
    # Create a dummy DataFrame if load fails, just for demonstrating the SQL
    print("Using a dummy DataFrame for SQL demonstration.")
    data = {
        'Month': ['Jan', 'Jan', 'Feb', 'Feb', 'Mar'],
        'Business_Area': ['HR', 'IT', 'HR', 'IT', 'Ops'],
        'Country': ['USA', 'USA', 'CAN', 'USA', 'GER'],
        'Cost_Element': ['Software', 'Hardware', 'Software', 'Services', 'Services'],
        'IT_Area': ['ERP', 'Networking', 'Security', 'ERP', 'Infrastructure'],
        'Planned': [100000, 50000, 120000, 60000, 80000],
        'Forecasted': [110000, 60000, 100000, 65000, 90000],
        'Actual': [130000, 75000, 90000, 100000, 70000]
    }
    df = pd.DataFrame(data)

# Print the first few rows and info to confirm structure
print("\n--- DataFrame Head ---")
print(df.head())
print("\n--- DataFrame Info ---")
df.info()

# --------------------------------------------------------------------------------
# Step 2: Setup In-Memory SQLite Database
# --------------------------------------------------------------------------------

# Create a connection to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Load the pandas DataFrame into a SQL table named 'IT_SPENDING'
# Note: table names are case sensitive in SQLite, but we use uppercase for clarity.
df.to_sql('IT_SPENDING', conn, if_exists='replace', index=False)
print("\nDataFrame loaded into SQLite table 'IT_SPENDING'.")

# --------------------------------------------------------------------------------
# Step 3: SQL Variance Analysis Queries
# --------------------------------------------------------------------------------

def run_query(query_title, sql_query):
    """Utility function to run a query and print results clearly."""
    print(f"\n\n========================================================")
    print(f"Query: {query_title}")
    print(f"========================================================\n")
    print(sql_query)
    try:
        result_df = pd.read_sql_query(sql_query, conn)
        print("\n--- RESULTS ---")
        print(result_df)
    except Exception as e:
        print(f"\n--- ERROR ---")
        print(f"SQL Error: {e}")


# --------------------------------
# Query 1: Comprehensive Variance Analysis
# Goal: Calculate all variances aggregated by all key dimensions.
# This query directly addresses the need to understand where discrepancies occur.
# --------------------------------
query_1_title = "1. Comprehensive Variance Analysis by All Dimensions"
sql_q1 = """
SELECT
    Month,
    Business_Area,
    Country,
    Cost_Element,
    IT_Area,
    SUM(Planned) AS Total_Planned,
    SUM(Forecasted) AS Total_Forecasted,
    SUM(Actual) AS Total_Actual,

    -- Variance 1: Actual vs Planned (A-P)
    SUM(Actual) - SUM(Planned) AS Variance_Actual_vs_Planned,

    -- Variance 2: Actual vs Forecasted (A-F)
    SUM(Actual) - SUM(Forecasted) AS Variance_Actual_vs_Forecasted,

    -- % Variance (Actual vs Planned) - Critical for relative context
    CASE
        WHEN SUM(Planned) = 0 THEN NULL
        ELSE ROUND(((SUM(Actual) - SUM(Planned)) * 100.0) / SUM(Planned), 2)
    END AS Pct_Variance_Actual_vs_Planned
FROM
    IT_SPENDING
GROUP BY
    Month,
    Business_Area,
    Country,
    Cost_Element,
    IT_Area
ORDER BY
    ABS(Variance_Actual_vs_Planned) DESC -- Order by largest absolute variance
LIMIT 10; -- Showing top 10 rows for brevity
"""
run_query(query_1_title, sql_q1)


# --------------------------------
# Query 2: Top 5 IT Areas with Highest Overspending (Actual > Planned)
# Goal: Pinpoint the biggest areas of concern (over budget).
# --------------------------------
query_2_title = "2. Top 5 IT Areas with Highest Overspending (Actual > Planned)"
sql_q2 = """
SELECT
    IT_Area,
    SUM(Planned) AS Total_Planned,
    SUM(Actual) AS Total_Actual,
    SUM(Actual) - SUM(Planned) AS Overspending_Variance
FROM
    IT_SPENDING
GROUP BY
    IT_Area
HAVING
    Overspending_Variance > 0 -- Filters for cases where Actual > Planned (Over budget)
ORDER BY
    Overspending_Variance DESC
LIMIT 5;
"""
run_query(query_2_title, sql_q2)

# --------------------------------
# Query 3: Monthly Trend Analysis (Actual vs Forecasted)
# Goal: Understand how expenditure control changes over time (by month).
# --------------------------------
query_3_title = "3. Monthly Trend of Spending vs. Forecast"
sql_q3 = """
SELECT
    Month,
    SUM(Planned) AS Total_Planned,
    SUM(Forecasted) AS Total_Forecasted,
    SUM(Actual) AS Total_Actual,
    -- Variance A vs F
    SUM(Actual) - SUM(Forecasted) AS Variance_Actual_vs_Forecasted,
    -- % Variance A vs F
    CASE
        WHEN SUM(Forecasted) = 0 THEN NULL
        ELSE ROUND(((SUM(Actual) - SUM(Forecasted)) * 100.0) / SUM(Forecasted), 2)
    END AS Pct_Variance_Actual_vs_Forecasted
FROM
    IT_SPENDING
GROUP BY
    Month
ORDER BY
    -- Assuming Month is an ordered string (e.g., '01-Jan', '02-Feb' etc.)
    Month;
"""
run_query(query_3_title, sql_q3)


# Close the database connection
conn.close()
print("\nAnalysis complete and database connection closed.")

Error loading data. Please check the CSV URL and permissions: HTTP Error 400: Bad Request
Using a dummy DataFrame for SQL demonstration.

--- DataFrame Head ---
  Month Business_Area Country Cost_Element         IT_Area  Planned  \
0   Jan            HR     USA     Software             ERP   100000   
1   Jan            IT     USA     Hardware      Networking    50000   
2   Feb            HR     CAN     Software        Security   120000   
3   Feb            IT     USA     Services             ERP    60000   
4   Mar           Ops     GER     Services  Infrastructure    80000   

   Forecasted  Actual  
0      110000  130000  
1       60000   75000  
2      100000   90000  
3       65000  100000  
4       90000   70000  

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Month          5 non-null      object
 1   Business_A